In [1]:
import math
import os.path
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
import numpy as np
import tensorflow as tf
# calculate accuracy
from sklearn import metrics

In [2]:
f_predpath = open('./Predicate_paths/city_capitals_1.csv','r')
f_predpath = f_predpath.readlines()
f_predpath = f_predpath[1:]
f_predpath = [f_predpath[i].rstrip('\n').split(',')[1:] for i in range(len(f_predpath))]
for i in range(len(f_predpath)):
    for j in range(len(f_predpath[0])):
        if (f_predpath[i][j] == 'TRUE'): 
            f_predpath[i][j] = 1
        if (f_predpath[i][j] == 'FALSE'):
            f_predpath[i][j] = 0
        f_predpath[i][j] = int(f_predpath[i][j])
train_predpath = np.array(f_predpath)


f_triple = open('./data_id/city_capital.csv','r')
f_triple  = f_triple.readlines()
f_triple = f_triple[1:]
f_triple = [f_triple[i].rstrip('\n').split(',')[:-1] for i in range(len(f_triple))]
for i in range(len(f_triple)):
    for j in range(len(f_triple[0])):
        f_triple[i][j] = int(f_triple[i][j])
f_triple = np.array(f_triple)
hrt_triples = f_triple

n_predicate = train_predpath.shape[1]-1
print("N_TRAIN_TRIPLES: %d" % train_predpath.shape[0])

N_TRAIN_TRIPLES: 259


In [3]:
# Parameters
learning_rate = 0.1
training_epochs = 15
display_step = 1

# Network Parameters
n_hidden_1 = 100 # 1st layer number of neurons
#n_hidden_2 = 1 # 2nd layer number of neurons
n_input =  n_predicate
n_classes = 2

embed_dim = n_hidden_1
bound = 6 / math.sqrt(n_hidden_1)

In [4]:
# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_uniform([n_input, n_hidden_1], minval=-bound, maxval=bound, seed=345)),
    #'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_uniform([n_hidden_1, n_classes], minval=-bound, maxval=bound, seed=345))
}
biases = {
    'b1': tf.Variable(tf.zeros([n_hidden_1])),
    #'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.zeros([n_classes]))
}


# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    #layer_1 = tf.matmul(x, weights['h1'])
    # Hidden fully connected layer with 256 neurons
    #layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_1, weights['out'])  + biases['out']
    #out_layer = tf.matmul(layer_1, weights['out']) 
    return out_layer

def trained_embedding(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    return layer_1
# Construct model
logits = multilayer_perceptron(X)
trained_new = trained_embedding(X)
    
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

In [11]:
with tf.Session() as session:
    #tf.global_variables_initializer().run()
    kf = KFold(n_splits=10, shuffle = True, random_state=233)
    print("MLP Neural Net: 10-folds Evaluation")
    i_fold = 1
    Confusion_mat = np.zeros([2,2])
    auc_score = 0.0
    for i_train, i_test in kf.split(train_predpath):
        #print("Fold ", i_fold)  
        tf.global_variables_initializer().run()
        train_predicates = np.array(train_predpath)[i_train]
        test_predicates = np.array(train_predpath)[i_test]
        train_tiples = np.array(hrt_triples)[i_train]    
        #print("Label 1: ",np.sum(test_predicates[:,0])," -- Label 0: ", len(test_predicates[:,0])-np.sum(test_predicates[:,0]))
        for epoch in range(training_epochs):

            accu_loss = 0.
            ninst = 0                 
            #print("Minibatches training... iteration: ", n_iter)           
            head_unique = np.unique(train_tiples[:,0])
            for i_head in head_unique:
                X_batch = train_predicates[train_tiples[:,0]==i_head,1:]
                tmp = train_predicates[train_tiples[:,0]==i_head,0]
                Y_labels = np.zeros([X_batch.shape[0], n_classes])
                for j in range(X_batch.shape[0]):
                    Y_labels[j,tmp[j]] = 1.
                _, c = session.run([train_op, loss_op], feed_dict = 
                                   {X: X_batch, Y: Y_labels})
                #accu_loss += l
            #print("Loss ", accu_loss)
        i_fold = i_fold + 1
        
        pred = tf.nn.softmax(logits)  # Apply softmax to logits
        y_p = tf.argmax(pred, 1)
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        tmp_ = test_predicates[:,0]
        test_labels = np.zeros([len(tmp_), n_classes])
        for j in range(len(tmp_)):
            test_labels[j,tmp_[j]] = 1.
        #print("Accuracy:", accuracy.eval({X: test_predicates[:,1:], Y: test_labels}))
        y_pred = session.run([y_p], feed_dict={X: test_predicates[:,1:]})[0]
        pred_proba = session.run([pred], feed_dict={X: test_predicates[:,1:]})[0]
        y_true = np.argmax(test_labels,1)
        auc_score += metrics.roc_auc_score(test_predicates[:,0], pred_proba[:, 1])
        Confusion_mat += metrics.confusion_matrix(test_predicates[:,0], y_pred)
    print "---Confusion Matrix---"
    print(Confusion_mat)
    print "---AUROC---"
    print auc_score/10

MLP Neural Net: 10-folds Evaluation
---Confusion Matrix---
[[ 195.   14.]
 [  13.   37.]]
---AUROC---
0.851362175201


In [13]:
    kf = KFold(n_splits=10, shuffle = True, random_state=233)
    print("Logistic Regression: 10-folds Evaluation")
    i_fold = 1
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    Confusion_mat = np.zeros([2,2])
    auc_score = 0.0
    for i_train, i_test in kf.split(train_predpath):
        #print("Fold ", i_fold)  
        train_predicates = np.array(train_predpath)[i_train]
        test_predicates = np.array(train_predpath)[i_test]
        train_tiples = np.array(hrt_triples)[i_train]    
        #print("Label 1: ",np.sum(test_predicates[:,0])," -- Label 0: ", len(test_predicates[:,0])-np.sum(test_predicates[:,0]))

        logistic = LogisticRegression()
        logistic.fit(train_predicates[:,1:], train_predicates[:,0])
        #print logistic
        i_fold = i_fold + 1
        
        #print(metrics.roc_auc_score(test_predicates[:,0], logistic.predict_proba(test_predicates[:,1:])[:,1]))
        auc_score += metrics.roc_auc_score(test_predicates[:,0], logistic.predict_proba(test_predicates[:,1:])[:, 1])
        Confusion_mat += metrics.confusion_matrix(test_predicates[:,0], logistic.predict(test_predicates[:,1:]))
    print "---Confusion Matrix---"
    print(Confusion_mat)
    print "---AUROC---"
    print auc_score/10
                                                    


Logistic Regression: 10-folds Evaluation
---Confusion Matrix---
[[ 200.    9.]
 [  18.   32.]]
---AUROC---
0.844290617849


In [ ]:
# Parameters
learning_rate = 0.1
training_epochs = 100
display_step = 1

# Network Parameters
n_hidden_1 = 80 # 1st layer number of neurons
#n_hidden_2 = 1 # 2nd layer number of neurons
n_input =  n_predicate
n_classes = 2

embed_dim = n_hidden_1
bound = 6 / math.sqrt(n_hidden_1)

# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_uniform([n_input, n_hidden_1], minval=-bound, maxval=bound, seed=345)),
    #'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_uniform([n_hidden_1, n_classes], minval=-bound, maxval=bound, seed=345))
}
biases = {
    'b1': tf.Variable(tf.zeros([n_hidden_1])),
    #'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.zeros([n_classes]))
}


# Create model
def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    #layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.matmul(x, weights['h1'])
    # Hidden fully connected layer with 256 neurons
    #layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    #out_layer = tf.matmul(layer_1, weights['out'])  + biases['out']
    out_layer = tf.matmul(layer_1, weights['out']) 
    return out_layer

def trained_embedding(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    return layer_1
# Construct model
logits = multilayer_perceptron(X)
trained_new = trained_embedding(X)
    
# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as session:
    #tf.global_variables_initializer().run()
    kf = KFold(n_splits=10, shuffle = True, random_state=233)
    print("MLP Neural Net: 10-folds Evaluation")
    i_fold = 1
    Confusion_mat = np.zeros([2,2])
    auc_score = 0.0
    for i_train, i_test in kf.split(train_predpath):
        #print("Fold ", i_fold)  
        tf.global_variables_initializer().run()
        train_predicates = np.array(train_predpath)[i_train]
        test_predicates = np.array(train_predpath)[i_test]
        train_tiples = np.array(hrt_triples)[i_train]    
        #print("Label 1: ",np.sum(test_predicates[:,0])," -- Label 0: ", len(test_predicates[:,0])-np.sum(test_predicates[:,0]))
        for epoch in range(training_epochs):

            accu_loss = 0.
            ninst = 0                 
            #print("Minibatches training... iteration: ", n_iter)           
            head_unique = np.unique(train_tiples[:,0])
            for i_head in head_unique:
                X_batch = train_predicates[train_tiples[:,0]==i_head,1:]
                tmp = train_predicates[train_tiples[:,0]==i_head,0]
                Y_labels = np.zeros([X_batch.shape[0], n_classes])
                for j in range(X_batch.shape[0]):
                    Y_labels[j,tmp[j]] = 1.
                _, c = session.run([train_op, loss_op], feed_dict = 
                                   {X: X_batch, Y: Y_labels})
                #accu_loss += l
            #print("Loss ", accu_loss)
        i_fold = i_fold + 1
        
        pred = tf.nn.softmax(logits)  # Apply softmax to logits
        y_p = tf.argmax(pred, 1)
        correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        tmp_ = test_predicates[:,0]
        test_labels = np.zeros([len(tmp_), n_classes])
        for j in range(len(tmp_)):
            test_labels[j,tmp_[j]] = 1.
        #print("Accuracy:", accuracy.eval({X: test_predicates[:,1:], Y: test_labels}))
        y_pred = session.run([y_p], feed_dict={X: test_predicates[:,1:]})[0]
        pred_proba = session.run([pred], feed_dict={X: test_predicates[:,1:]})[0]
        y_true = np.argmax(test_labels,1)
        auc_score += metrics.roc_auc_score(test_predicates[:,0], pred_proba[:, 1])
        Confusion_mat += metrics.confusion_matrix(test_predicates[:,0], y_pred)
    print "---Confusion Matrix---"
    print(Confusion_mat)
    print "---AUROC---"
    print auc_score/10

MLP Neural Net: 10-folds Evaluation
